In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
train_dir ='/content/drive/MyDrive/data/train'
test_dir='/content/drive/MyDrive/data/test'

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

In [6]:
img_width,img_height =256,256
input_shape=(img_width,img_height,3)
batch_size = 24
train_generator =train_datagen.flow_from_directory(train_dir,
                                                   shuffle=True,
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)

Found 3742 images belonging to 38 classes.
Found 380 images belonging to 38 classes.


In [7]:
INCEPTIONV3_TFHUB = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'

In [8]:
num_classes = train_generator.num_classes

# fetch the feature extractor from the tf_hub
feature_extractor = hub.KerasLayer(INCEPTIONV3_TFHUB, input_shape=input_shape)

# make the feature extractor trainable
feature_extractor.trainable = True

# create the sequential model
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 38)                77862     
Total params: 21,880,646
Trainable params: 21,846,214
Non-trainable params: 34,432
_________________________________________________________________


In [10]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
)

Epoch 1/10
155/155 [==============================] - 477s 3s/step - loss: 0.5005 - accuracy: 0.9306 - val_loss: 0.5325 - val_accuracy: 0.9079
Epoch 2/10
155/155 [==============================] - 40s 260ms/step - loss: 0.3776 - accuracy: 0.9691 - val_loss: 0.4335 - val_accuracy: 0.9447
Epoch 3/10
155/155 [==============================] - 41s 267ms/step - loss: 0.3121 - accuracy: 0.9898 - val_loss: 0.4282 - val_accuracy: 0.9474
Epoch 4/10
155/155 [==============================] - 41s 267ms/step - loss: 0.3008 - accuracy: 0.9925 - val_loss: 0.3419 - val_accuracy: 0.9737
Epoch 5/10
155/155 [==============================] - 42s 267ms/step - loss: 0.2905 - accuracy: 0.9960 - val_loss: 0.3795 - val_accuracy: 0.9632
Epoch 6/10
155/155 [==============================] - 43s 274ms/step - loss: 0.2853 - accuracy: 0.9970 - val_loss: 0.3732 - val_accuracy: 0.9711
Epoch 7/10
155/155 [==============================] - 42s 271ms/step - loss: 0.2835 - accuracy: 0.9978 - val_loss: 0.3479 - val_accu

In [13]:
# evaluate the model
loss, accuracy = model.evaluate(test_generator)
# train accuracy
train_loss, train_accuracy = model.evaluate(train_generator)

156/156 [==============================] - 13s 86ms/step - loss: 0.2703 - accuracy: 0.9984


In [15]:
"Trained Model for {} epochs, train accuracy: {:5.2f}%, test accuracy: {:5.2f}%".format(4, 100*train_accuracy, 100*accuracy)

'Trained Model for 4 epochs, train accuracy: 99.84%, test accuracy: 97.37%'

In [ ]:
model.save('leafDiseaseDetect2.h5')